<a href="https://colab.research.google.com/github/AliBavarchee/MAPD_II-Analysis-of-COVID19-PAPERS/blob/main/Final_assignment_of_MAPD_B_Part_I_Analysis_of_COVID19_papers_Ali_Bavarchee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment of Management and Analysis of Physics Datasets (MOD B) Part I

## Analysis of Covid-19 papers

## Ali Bavarchee

In [ ]:
import numpy as np
import pandas as pd
import dask
import sklearn
import matplotlib.pyplot as plt
import time
import json
from dask.distributed import Client, LocalCluster, progress
from collections import defaultdict, Counter, namedtuple
import nltk
from nltk.corpus import stopwords
import dask.bag as db
import dask.dataframe as dd
from itertools import islice
from operator import *
from tqdm import tqdm
import bokeh.palettes as palette
import seaborn as sn
import gensim
from gensim.models import *
import fasttext
import warnings
import os
import io
import re

## 3.1 Word counter distributed algorithm

In [ ]:
os.getcwd()

In [ ]:
os.chdir('/home/ali/Documents/MPDAPB/Final Assignments/Distribution')
os.getcwd()

In [ ]:
json_str = {"paper_id", "metadata", "title", "authors", "first", "middle", "last", "suffix", "affiliation", "email", "abstract", "text", "cite_spans"}

In [ ]:
%%bash
mkdir -p data/papers_in_json_singleline
cd data/papers_in_json
for filename in *.json; do
    echo $filename
    jq -c . $filename > ../papers_in_json_singleline/$filename
done

In [ ]:
nltk.download()

In [ ]:
client = Client()
client

In [ ]:
# Import Data and read them by Dask.Bag after form them in single line
DB = db.read_text('data/papers_in_json_singleline/*.json').map(json.loads).repartition(npartitions=10)

In [ ]:
DB

In [ ]:
# Collect the Dictionary
def join(txt):
    text=''
    for rec in txt:
        text+=rec['text']
    return text
 
t3xt = DB.pluck("body_text").map(join)
t3xt.take(1)

In [ ]:
# Merging all pieces of Data
def joiner(text):
    return ''.join([item['text'] for lines in item['body_text']])

# filter
# defining some routine words and cliping
rm_word = ['also', 'may', 'et' , 'using'
           , 'used', 'al', 'two', 'one', 'eg', 'could', 'use']
# Stop_words subpackage of Natural Language toolkit
naturalang = stopwords.words('english') + rm_word
# imlimenting stop words
stop_words = re.compile(r'\b(' + r'|'.join(naturalang) + r')\b\s*')
# symbols refer to footnotes, citations and references
appdix = re.compile('\[[\d]+\]')
# transforming all the words of dataset to lowercase (or caps) to prevent duplication
lowecase  = s.lower()
# Numbers
Num = re.compile('((^|\s)[\d]+)')
# another defenition of numbers and digits
Numbeer = re.compile(r'\b[0-9]+\b\s*')
# Punctuation marks
punk  = re.compile('([\.,:?;!](?=\w))')
# websites and emails and symbols belong to social  network
www = re.compile('(http(s)?:\/\/.)?(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)')
# Patterns
patterns = re.compile('[^a-zA-Z\d\s]')
# Space and blanks
dash = re.compile('\s([\s]+)')


In [ ]:
# creating a function for cleaning the text from all rm_words, stop words, punctuation marks etc.
def sanitization(txt):
    sanitization = txt.lower()
    sanitization = punk.sub(' ', sanitization)
    
    sanitization = www.sub('', sanitization)
    sanitization = Num.sub('', sanitization)
    sanitization = patterns.sub('', sanitization)
    sanitization = appdix.sub('', sanitization)
    sanitization = dash.sub('', sanitization)
    sanitization = stop_words.sub('', sanitization)
    sanitization = Numbeer.sub('', sanitization)
    sanitization = sanitization.split()
    return sanitization

In [ ]:
# Executing "sanitization" function to reach -to some extend a clean data set 
sanit_txt = t3xt.map(sanitization)
sanit_txt.take(1)

In [ ]:
# a Func to count each word of data
def W_counter(txt):
    counts = dict(zip(Counter(txt).keys(), Counter(txt).values()))
    wlist = [{"word":i , "num": j} for i,j in counts.items()]
    return wlist
 
WC = sanit_txt.map(W_counter)
WC.take(1)

In [ ]:
# Accumulating all the words through out the data by using foldby
TWC = WC.flatten().foldby('word', binop=Word_Conter, initial=0, combine=add, combine_initial=0)

In [ ]:
sum_tot = TWC.compute()

In [ ]:
sum_tot

In [ ]:
# Trying to sort list, but it returns latter part of set (Z)
sum_total_sort_bottom = sorted(sum_tot,key=itemgetter(0),reverse=True)

In [ ]:
sum_total_sort_bottom

In [ ]:
sum_total_sort_v2 = sorted(sum_tot, key=itemgetter(1), reverse=True)

In [ ]:
sum_total_sort_v2

In [ ]:
stsv = sum_total_sort_v2

In [ ]:
n = 20 # Twenty of the most repeatitive words in the COVID19 papers ; the number 20 is arbitary

Top_Words = dict( islice(stsv, n))

figure, ax = plt.subplots(figsize=(20, 15))

ax.barh(list(Top_Words.keys())[::-1], list(Top_Words.values())[::-1],
        color=palette.plasma(n))
        
ax.set_xlabel('Total Number of Word', fontsize=22)
ax.set_ylabel('Words', fontsize=22)
ax.set_title('The Highest Frequency Words', fontsize=32)
plt.show()

In [ ]:
%%time
timing = TWC.compute()
tempo = stsv


In [ ]:
print(tempo)

In [ ]:
print(timing, 'ms')

In [ ]:
client.close()
cluster.close()

In [ ]:
# Grid search

def Wtimming(n_workers, n_partitions):
    cluster = LocalCluster(n_workers = n_workers)
    client = Client(cluster) 
    camin = time.time()
    
    DB = db.read_text('data/papers_in_json_singleline/*.json').map(json.loads).repartition(npartitions=n_partitions)
    DB = DB.map(joiner).map(sanitization).map(W_counter)
       
    TWCT = WC.flatten().foldby('word', binop=Word_Conter, initial=0, combine=add, combine_initial=0)
    sum_tot_T = TWCT.compute()
    st_sorted_T = sorted(sum_tot_T, key=itemgetter(1), reverse=True)
    
    fine = time.time()
    
    client.close()
    cluster.close() 
    
    interval_T = fine - camin
    
    return interval_T

In [ ]:
workr = [1, 2, 4, 5, 8, 12]
partn = [1, 2, 5, 10, 23, 100]

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# grid search for examination of timing with various number of worker/partition
tim_wp = []

for w in tqdm(workr):
    for p in tqdm(partn):
        t = Wtimming(n_workers = w, n_partitions = p*w)
        tim_wp.append(t)
        print("Workers=", w, "Partitions=", p*w, "Time=", t,"s")

In [ ]:
plot_time = np.array(tim_wp).reshape(len(workr), len(partn))

df_Time = pd.DataFrame(plot_time)
plt.figure(figsize=(20,16))
ax = sn.heatmap(df_Time, annot=True, annot_kws={"size": 22},fmt=".3g",
                xticklabels=partn, yticklabels=workr,
                cbar_kws={'label':'Execution time [s]'},cmap=sn.color_palette("Accent",50) )
ax.invert_yaxis()
ax.set_xlabel("Partitions per Worker", fontsize=22, fontweight='bold')
ax.set_ylabel("Number of Workers", fontsize=22, fontweight='bold')

plt.show()

In [ ]:
print('\033[01mAmong the set of diffrent number of workers and partitions, the cluster with 5 workers and 10 partitions per worker exxecuted faster than other. Worker = 1 and partition = 100 take the longest time. ')

In [ ]:
client.close()

# 3.2 - The worst and best represented countries in the research

In [ ]:
cluster = LocalCluster(n_workers=4)
client = Client(cluster)

In [ ]:
# import Data.json by Dask.Bag
filename = os.path.join('data', 'papers_in_json_singleline', '*.json')
lines = db.read_text(filename)
DBJ = lines.map(json.loads).repartition(10)

In [ ]:
# Selecting and extracting metadata(authors) by pluck (serie)
authors = DBJ.pluck('metadata').pluck('authors')
authors.take(1)

In [ ]:
# Another way to select "authors" from all dictioneries of the data (bag.dataframe)
authors = DB.pluck('metadata').pluck('authors')

In [ ]:
# Import Data and read them by Dask.Bag after form them in single line
DB = db.read_text('data/papers_in_json_singleline/*.json').map(json.loads).repartition(npartitions=10)

In [ ]:
# flattening metadata for authors to form a dag dataframe
def flattening(record):
    Uni=Nat=Lab=''
    if 'institution' in record['affiliation'].keys():
        Uni=record['affiliation']['institution']
    else: Uni=np.nan
    
    if 'laboratory' in record['affiliation'].keys():
        Lab=record['affiliation']['laboratory']
    else: Lab=np.nan
    
    if 'location' in record['affiliation'].keys():
        if 'country' in record['affiliation']['location'].keys():
            Nat=record['affiliation']['location']['country']
        else: Nat="N/A"
    else: Nat=np.nan
    
    if Uni=='': Uni=np.nan
    if Lab=='': Lab=np.nan
    if Nat=='': Nat=np.nan
    
    return {
        'name': record['first'],
        #'middle name': record['middle'], 
        'last name': record['last'],
        'Institute':  Uni,
        'Laboratory': Lab,
        'Country': Nat,
    }
authors.flatten().map(flattening).take(10)

In [ ]:
# Converting Bag to Dataframe
df_author = authors.flatten().map(flattening).to_dataframe()
df_author.tail(10)

In [ ]:
len(df_author)

In [ ]:
# counting the number of repetition of each country in data
paesi = df_author.Country.value_counts()
# Droping out the missing values in dask.bag serie with dropna
countries_cdf = paesi.dropna()

In [ ]:
ccdf = countries_cdf

In [ ]:
ccdf.describe

In [ ]:
ccdf.describe()

In [ ]:
%%time

ccdf = ccdf.compute().sort_values(ascending=False)

In [ ]:
rmnacountries = paesi.dropna()

In [ ]:
rmnacountries.describe

In [ ]:
countrieZ = rmnacountries.compute().sort_values(ascending=False)

In [ ]:
n = 14 # fourteen countries with highest rate of references from COVID19 data

plt.figure(figsize=(21,17))

X = np.arange(n)

ax = plt.axes()
ax.set_xticks(X)
ax.set_xticklabels(countrieZ.index[:n])

plt.bar(X, countrieZ.values[:n], color='olivedrab')
plt.xlabel('Country', size=22)
plt.ylabel('Frequency', size=22)
plt.title('The Best represented countries in Covid-19 papers', size=26)
plt.show()

In [ ]:
print("\033[1m\033[95mTop presented countries")
display(countrieZ.head(33))

In [ ]:
Worst = countrieZ.tail(40)

In [ ]:
Worst

In [ ]:
n = 10 # plotting ten countries with the laest reference from COVID19 Data
Worst = countrieZ.tail(n)
plt.figure(figsize=(23,4))
 
X = np.arange(n)
    # gives histogram aspect to the bar diagram
 
ax = plt.axes()
ax.set_xticks(X)
ax.set_xticklabels(Worst.index[:n])
 
plt.bar(X, Worst.values[:n], color='b')
plt.xlabel('Country', size=22)
plt.ylabel('Frequency', size=22)
plt.title('The Worst represented countries in Covid-19 papers', size=26)
plt.show()

In [ ]:
ccdf.describe

### The Best and Worst affiliations(universities)

In [ ]:
%%time
Unii = df_author.Institute.value_counts().compute()
Unii = Uni.dropna() 

In [ ]:
Unii

In [ ]:
n = 5 # Plot 5 academic centers that published the most data(papers, surveys & ...)

plt.figure(figsize=(21,25))

X = np.arange(n)

ax = plt.axes()
ax.set_xticks(X)
ax.set_xticklabels(Unii.index[1:n])

plt.bar(X, Unii.values[:n], color='orange')
plt.xlabel('Institute', size=22)
plt.ylabel('Frequency', size=22)
plt.title('The outstanding institutes in Covid-19 papers', size=26)
plt.show()

In [ ]:
Unii.tail(20)

In [ ]:
n = 10 # The worst institues based on the data
Worsinst = Unii.tail(n)

plt.figure(figsize=(25,9))

X = np.arange(n)

ax = plt.axes()
ax.set_xticks(X)
ax.set_xticklabels(Worsinst.index[1:n])

plt.bar(X, Worsinst.values[:n], color='black')
plt.xlabel('Institute', size=22)
plt.ylabel('Frequency', size=22)
plt.title('The passive institutes in data', size=26)
plt.show()

### Different number of partitions and workers

In [ ]:
# A func for computing times of parallel computing!!! funcution as usual works owing to iteration
def timming_countries(n_workers, n_partitions):
    cluster = LocalCluster(n_workers = n_workers)
    client = Client(cluster) 

    start = time.time()
    
    DB = db.read_text('data/papers_in_json_singleline/*.json').map(json.loads).repartition(npartitions=10)
    authors = DB.pluck('metadata').pluck('authors')
    df_author = authors.flatten().map(flattening).to_dataframe()
    
    countriesC = df_author.Country.value_counts()
    
    fin = time.time()
    
    client.close()
    cluster.close() 
    
    Time_interval = fin - start
    
    return Time_interval

In [ ]:
workr = [1, 2, 4, 5, 12]
partn = [1, 2, 5, 10, 23, 100]

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Same as previous - Gread search - to compare timming of parallel running of codes with defferent number of workers*& partitions
Ctimeset =[]
for w in tqdm(workr):
    for p in tqdm(partn):
        t = timming_countries(n_workers = w, n_partitions = p*w)
        Ctimeset.append(t)
        print("Workers=", w, "Partitions=", p*w, "Time=", t,"s")

In [ ]:
plot_time_countries = np.array(Ctimeset).reshape(len(workr), len(partn))

df_Time_countries = pd.DataFrame(plot_time_countries)
plt.figure(figsize=(20,16))
ax = sn.heatmap(df_Time_countries, annot=True, annot_kws={"size": 22},fmt=".3g",
                xticklabels=partn, yticklabels=workr,
                cbar_kws={'label':'Execution time [s]'},cmap=sn.color_palette("inferno",50) )
ax.invert_yaxis()
ax.set_xlabel("Partitions per Worker", fontsize=22, fontweight='bold')
ax.set_ylabel("Number of Workers", fontsize=22, fontweight='bold')

plt.show()

Surprisingly the best timming is granted by cluster with 2 workers and 2 partitions for each worker, and the longest is for 12 workers and 12 partitions.

In [ ]:
=========================================================

In [ ]:
workr = [1, 2, 4, 5, 8, 12]
partn = [1, 2, 5, 10, 23, 100]


In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
Ctimesett =[]
for w in tqdm(workr):
    for p in tqdm(partn):
        t = timming_universities(n_workers = w, n_partitions = p*w)
        Ctimesett.append(t)
        print("Workers=", w, "Partitions=", p*w, "Time=", t,"s")

In [ ]:
plot_time_inst = np.array(Ctimesett).reshape(len(workr), len(partn))

df_Time_inst = pd.DataFrame(plot_time_inst)
plt.figure(figsize=(20,16))
ax = sn.heatmap(df_Time_inst, annot=True, annot_kws={"size": 22},fmt=".3g",
                xticklabels=partn, yticklabels=workr,
                cbar_kws={'label':'Execution time [s]'},cmap=sn.color_palette("plasma",50) )
ax.invert_yaxis()
ax.set_xlabel("Partitions per Worker", fontsize=22, fontweight='bold')
ax.set_ylabel("Number of Workers", fontsize=22, fontweight='bold')

plt.show()

The optimum number of the worker has been 4 with 400 partitions.
The client with one worker has shown the slowest preformance.

In [ ]:
df_Time_inst

In [ ]:
client.close()

## 3.3 Get the embedding for the title of the papers
This method of data analysis is useful to understand a general picture of titles as in this representation of data, the words with common similarities, same meaning, root of the words etc. will be revealed and the relation between words will be emanated.

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import fasttext
import fasttext.util

In [ ]:

ft = fasttext.load_model('cc.en.300.bin')
ft.save_model('cc.en.300.bin')

In [ ]:
ft = fasttext.load_model('cc.en.300.bin')

In [ ]:
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

In [ ]:
def load_wordlist(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    embeddings = np.zeros((n, d), dtype=np.float32)
    data ={}
    for i, line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = list(map(float, tokens[1:]))
    return data
model = KeyedVectors.load_word2vec_format('/home/ali/Documents/FADistribution/wiki.en.vec')
#model['hello']

In [ ]:
# getting the train dataset by genism and computing the time of execution
start=time.time()

Vword = KeyedVectors.load_word2vec_format('wiki.en.vec')

stop=time.time()

interval = stop - start
print(interval)

In [ ]:
# getting the train dataset by genism and computing the time of execution
start=time.time()

Vword = KeyedVectors.load_word2vec_format('/home/ali/Documents/FADistribution/wiki.en.vec')

stop=time.time()

interval = stop - start
print(interval)

In [ ]:
Vword['collection']

In [ ]:
client = Client()
client

In [ ]:
filename = os.path.join('data', 'papers_in_json_singleline', '*.json')
lines = db.read_text(filename)
DBJ = lines.map(json.loads).repartition(10)

In [ ]:
DB = db.read_text('data/papers_in_json_singleline/*.json').map(json.loads).repartition(npartitions=10)

In [ ]:
#wordz, embeddings = load_word2vec_format('/home/ali/Documents/FADistribution/wiki.en.vec',binary=True)


In [ ]:
def dictionary(record):
    return {
        "paper_id": record['paper_id'],
        "title": record['metadata']['title']
            }

In [ ]:
embeded = DB.map(dictionary).filter(lambda record: bool(record['title'])).compute()

In [ ]:
# funct makes title vectorized - 
def embedder(text, wiki):
    unipassage_split = text.lower().split()
    words_embed=[]

    for w in unipassage_split:
        try:
            words_embed.append( wiki[w])
        except:
            pass 
    return words_embed

In [ ]:
# A funct to make a dictionary of text based on ip paper and embedded titles by model
def dictionary(record, model):
    return{
        "paper_id": record["paper_id"],
        "title": embedder(record['title'], model)
    }

In [ ]:
# creating a dictionary by dictionary function
embedded_titlez = [create_dictionary(title,  Vword) for title in embeded]

In [ ]:
embedded_titlez

In [ ]:
emtitlez = DBJ.map(dictionary).filter(lambda record: bool(record['title'])).compute()

In [ ]:
emtitlez

In [ ]:
# transforming the list of embeded titles to dask.bag from python sequence
embedded_titles = db.from_sequence(embedded_titlez)
embedded_titles.take(4)

In [ ]:
embu = DB.map(dictionary).filter(lambda record: bool(record['title']))

## 3.3.1 Bonus point

In [ ]:
def cos_similarity(V, W):

    n_V = np.linalg.norm(V)   
    m_V =  np.linalg.norm(W)
    
    m = min(len(V), len(W)) 
    result = np.vdot(V[:m], W[:m])/(V*M)
    return result


def similarity(paper):
    V_embedding = paper["emtitlez"].flatten()
    V_embedding_length = len(V_embedding)
    similaritI = np.zeros(N_papers, dtype=np.float32)
    
    for i, p in enumerate(all_embeddings):
        W_embedding = p["emtitlez"].flatten()
        common_length = min(V_embedding_length, len(W_embedding))
        
        similarities[i] = cosine_similarity(V_embedding[:common_length],W_embedding[:common_length])
    
    return similarities

In [ ]:
%%time
similaritI = db.from_sequence(emtitlez).map(similarity)
similaritI


In [ ]:
%%time
similaritII = db.from_sequence(embedded_titlez).map(similarity)
similaritII


In [ ]:
%%time
similaritIII = db.from_sequence(embedded_titles).map(similarity)
similaritIII


In [ ]:
plot_cos_similarity  = np.array(similaritII)
#plot_cos_similarity = pd.DataFrame(plot_cos_similarity)
#plot_cos_similarity = plot_cos_similarity.dropna()
#cosine_similarities = pd.DataFrame(plot_time_inst)
plt.figure(figsize=(20,16))
ax = sn.heatmap(plot_cos_similarity, annot=True, annot_kws={"size": 22},fmt=".3f",
                cbar_kws={'label':'Execution time [s]'},cmap=sn.color_palette("plasma",50) )
ax.invert_yaxis()
ax.set_xlabel("Titles", fontsize=22, fontweight='bold')
ax.set_ylabel("Titles", fontsize=22, fontweight='bold')

plt.show()

In [ ]:
similarities = np.array(similaritI.compute()) 
#similarities[np.isnan(similaritI)] = 0 #remove NaNs
#similarities[np.diag_indices(len(similaritI))] = 0 #remove diagonal

In [ ]:
similarities = np.array(similaritIII) 

In [ ]:
similarities.compute()

In [ ]:
client.close()


In [ ]:
cluster.close()

In [ ]:
from plotfile import Plotter
import matplotlib.pyplot as plt

app = Plotter(xval=range(0,10), yval=range(0,10))
plot = app.plotthing()
app.show(plot)

In [ ]:
countriez = list(set(DB.pluck('metadata').pluck('authors').flatten()
                .filter(lambda record: bool(record['affiliation']))
                .filter(lambda record: bool(record['affiliation']['location']))
                .pluck('affiliation').pluck('location')
                .map(lambda record: record['country'] if 'country' in record.keys() else None).compute()))

In [ ]:
n = 2
Top_countries = dict( islice(countries_cdf, n))

figure, ax = plt.subplots(figsize=(20, 15))

ax.barh(list(countries_cdf.keys())[::-1], list(Top_countries.values())[::-1],
        color=palette.plasma(n))
        
ax.set_xlabel('Total Number of paper', fontsize=22)
ax.set_ylabel('Country', fontsize=22)
ax.set_title('The Highest words', fontsize=32)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
n = 21

ax.bar(countries_cdf.index[idx::-1], countries_cdf.values[n::-1],
        color=palette.cividis(n) )
ax.set_xlabel('Total', fontsize=14)
ax.set_title('Countries')

plt.show()

In [ ]:
universities = df_author.Institute.value_counts()

In [ ]:
def plot_top(data, n):
    data = data.nlargest(n+1).compute()
    data = data[data.axes[0]!='Unknown']
    height = [data[i] for i in range(0,n)]
    bars = [data.axes[0][i] for i in range(0,n)]
    y_pos = np.arange(len(bars))
    
    fig, ax = plt.subplots(figsize=(10,int(n*0.3)))
    index = n

    ax.barh(y_pos[:index], height[:index],color=palette.inferno(index))
    plt.yticks(y_pos[:index], bars[:index])
    ax.grid(True, which="both", ls="-",color='0.66')
    ax.set_axisbelow(True)

In [ ]:
titlezZ = DB.map(embeder)
title_embeddingZ = titlez.compute()
title_embeddingZ 

In [ ]:
plot_top(paesi, 20)

In [ ]:
universities = author.groupby('Institution').author.count().compute()
sorted_uni = universities.sort_values(ascending=False) # Sorting in descending order
sorted_uni = sorted_uni.drop("N/A")

In [ ]:
sort_df = paesi.sort_values(ascending=False)

In [ ]:
sort_df = df_author.sort_values(ascending=False) # Sorting in descending order
clean_df = sort_df.drop("Missing")

In [ ]:
Client.close()

In [ ]:
CumSum = sanit_txt.flatten().foldby('word', binop=dictionary, initial=0, combine=add, combine_initial=0).compute()

In [ ]:
def collactor(body_text):
    con = [item['text'] for item in body_text]
    text= "".join(con)
    text = re.sub('\W+',' ', text) 
    text = text.lower() 
    text = rm_word.sub('', text)
    vocabs = text.split()
    
    return vocabs


In [ ]:
%%time
tempo = mapp.compute()

In [ ]:
=======================================================================

In [ ]:
==========================================================